In [ ]:
# Author: Chandranil Chakraborttii
# Project: Learning I/O Access Patterns to Improve Prefetching in SSDs
# Paper Link :https://www.researchgate.net/profile/Chandranil_Chakraborttii/publication/344379801_Learning_IO_Access_Patterns_to_Improve_Prefetching_in_SSDs/links/5f6e28fba6fdcc00863adb13/Learning-I-O-Access-Patterns-to-Improve-Prefetching-in-SSDs.pdf
# Loading libraries

import csv
import sys
import pandas as pd
import os
import glob
from collections import Counter

# Loading all files
# Check coverage
# Update in output file


path  = r"/soe/hlitz/maxwell/notebooks/Prefetching_SSD/Data/output/DisplayAds"  
output  = r"/soe/hlitz/maxwell/notebooks/Prefetching_SSD/Data/output/DisplayAds" 
names = ['TimeStamp','Host_Name','DiskNumber','Operation_Type','ByteOffset','IOSize','Response_Time','DiskNum']


all_files = glob.glob(os.path.join(path, "displayads_total.csv"))
print(len(all_files))


Filename = []
Num_obs = []
Coverage_Offset_1000 = []
Coverage_Delta_1000 = []
Coverage_Delta2_1000 = []

count = 0
for file in all_files:
    count=count+1
    spl  = file.split("/")
    p = len(spl)
    name = spl[p-1]
    part_num = spl[p-2]
    print("Processing File Number :" + str(count))
#     print("Processing File Number :")
#     print(count)
    print("File Name :"  + str(name))
#     print(name)
    Filename.append(name)
  
    df = pd.read_csv(file,engine='python',index_col=False)
    df["ByteOffset"]= df["ByteOffset"].astype(float) 
    df['ByteOffset_Delta'] = df['ByteOffset'] - df['ByteOffset'].shift(-1)
    
    Num_obs.append(len(df))
    print("Total Number of Observations")
    print(len(df))
    
    x = Counter(df['ByteOffset'])
    vals = {}
    vals =  x.most_common(1000)
    bo_list = []
    coverage = 0
    for key,value in vals:
        bo_list.append(key)
        coverage = coverage + value
    print("Percentage Coverage : Offset   " + str((coverage/len(df))*100))
#     print((coverage/len(df))*100)
    Coverage_Offset_1000.append((coverage/len(df))*100)
    if ((coverage/len(df))*100  > 95.0):
        print("FLAGGED....!")
        print("FLAGGED....!")
        print("FLAGGED....!")
    
    
    

    x = Counter(df['ByteOffset_Delta'])
    vals = {}
    vals =  x.most_common(1000)
    bo_list = []
    coverage = 0
    for key,value in vals:
        bo_list.append(key)
        coverage = coverage + value
    

#     print("Percentage Coverage : Delta")
#     print((coverage/len(df))*100)
    print("Percentage Coverage : Delta   " + str((coverage/len(df))*100))
    Coverage_Delta_1000.append((coverage/len(df))*100)
    if ((coverage/len(df))*100  > 95.0):
        print("FLAGGED....!")
        print("FLAGGED....!")
        print("FLAGGED....!")
    
    
    df['ByteOffset_Delta'] = df['ByteOffset_Delta'].fillna(0)
    df['ByteOffset_Delta_2'] = df['ByteOffset_Delta'] - df['ByteOffset_Delta'].shift(-1)
    
    x = Counter(df['ByteOffset_Delta_2'])
    vals = {}
    vals =  x.most_common(1000)
    bo_list = []
    coverage = 0
    for key,value in vals:
        bo_list.append(key)
        coverage = coverage + value
    

#     print("Percentage Coverage : Delta 2")
#     print((coverage/len(df))*100)
    print("Percentage Coverage : Delta 2   " + str((coverage/len(df))*100))
    Coverage_Delta2_1000.append((coverage/len(df))*100)
    if ((coverage/len(df))*100  > 95.0):
        print("FLAGGED....!")
        print("FLAGGED....!")
        print("FLAGGED....!")
    
    
    print()
    print()
    print()

result_col = ['Filename','Num_obs','Coverage_Offset_1000','Coverage_Delta_1000']
result = pd.DataFrame(columns=result_col)
result['Filename'] = Filename
result['Num_obs'] = Num_obs
result['Coverage_Offset_1000'] = Coverage_Offset_1000
result['Coverage_Delta_1000'] = Coverage_Delta_1000
result['Coverage_Delta2_1000'] = Coverage_Delta2_1000

outname = part_num +'_coverage.csv'
filename_wr_out = os.path.join(output,outname)

result.to_csv(filename_wr_out,index=False)

In [ ]:
print("Done!!")